<a href="https://colab.research.google.com/github/tatyanach/work-projects/blob/main/write_txt_and_json_for_model_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import json
import ast
from pandas.api.types import is_number


1.   загрузили таблицы новой разметки по классам
2.   определили функцию, которая будет читать таблицы
3.   запустили цикл, который читает все таблицы в директории определенной нами функцией, записывает таблицы в один датафрейм, а также разбивает наименование таблицы и записывает в датафрейм класс, указанный в наименовании  


In [ ]:
def reading(data):
  df = pd.read_excel(f"{dir}{data}")
  return df
  
concat_df = pd.DataFrame(columns=["word", 'label',"sentence", "class"])

dir = '/content/'



In [ ]:
for file in os.listdir(dir):
  if file.endswith(".xlsx"):
    df = reading(file)

    separated = re.split("_|\.", file)
    df["class"] = separated[2]
    

    concat_df = pd.concat([df,concat_df], ignore_index = True)

In [ ]:
# сохраняем если нужно
# concat_df.to_csv("labels_and_classes_with marks.csv", encoding = "utf-8")

4. Убираем марки разметки (B-, I-) из колонки тегов




In [ ]:
concat_df['label'] = concat_df['label'].apply(lambda x: x.split("-")[1] if x != "O" else x)
concat_df['label']

In [ ]:
concat_df['label'] = concat_df['label'].apply(lambda x: x.replace(x, "коррозия(припуск/защита)") if x == "коррозия (припуск/защита)" else x)

In [ ]:
# сохраняем если нужно
concat_df.to_csv("labels_and_classes.csv", encoding = "utf-8")

**class_labels**

---



словарь, где ключ - номер класса, а значение - лист его тегов

In [ ]:
#группируем по классам и записывает все теги класса в лист
tags_df = concat_df.groupby('class')['label'].apply(list).reset_index(name='теги')

In [ ]:
#оставляем только уникальные значения тегов
tags_df['теги'] = tags_df['теги'].apply(lambda x: list(set(x)))
tags_df

In [ ]:
# сохраняем если необходимо
tags_df.to_csv("tags_new classes.csv", encoding = "utf-8")

In [ ]:
#создаем словарь, где ключ - номер класса, значение - лист его тегов
mydict = dict(zip(tags_df["class"], tags_df['теги']))
mydict

In [ ]:
#открываем старый текстовый файл
with open("/content/class_labels.txt", 'r') as f:
  data_txt = f.read()
result = ast.literal_eval(data_txt)

In [ ]:
result

In [ ]:
# записываем в старый текстовый файл недостающие новые данные
for key,value in mydict.items():
  if key not in result:
    result[key]= value
  else:
      for i in value:
          if i in result[key]:
            continue
          else:
              result[key].append(i)



Далее необходимо удалить ненужные уже теги, которые пришли со старым текстовым файлом. Запускаем цикл, который удаляет старые теги, если они присутствуют в словаре.

In [ ]:
deleted_tags = ["НД основной", "климатическое исполнение", "длина", 'ширина', 'высота', 'толщина'
                'Артикул', 'толщина покрытия', 'вес', 'вес/грузоподъемность', 'грузоподъемность', 'масса/объем', 
                'сезон/темп.режим', 'категория'
                ]

for key,value in result.items():
  for i in value:
    if i in deleted_tags:
      value.remove(i)

In [ ]:
result

сохраняем все в файлы


In [ ]:
result_txt = json.dumps(result, ensure_ascii=False)

with open("class_labels.txt", "w", encoding= 'utf-8') as text_file:
    text_file.write(result_txt)

In [ ]:
with open("class_labels.json", "w", encoding= "utf-8") as fp:
    json.dump(result, fp, ensure_ascii=False) 

**PRODUCT CLASS**


---
словарь, где значение - вид изделия, ключ - лист из классов, в которых изделие встречается


In [ ]:
concat_df = pd.read_csv("/content/labels_and_classes.csv")

# concat_df['label'] = concat_df['label'].apply(lambda x: x.split("-")[1] if x != "O" else x)

df = concat_df[["label","class","word"]]
df = df[df["label"] == 'Вид изделия']

In [ ]:
#чистим данные от символов отличных от русского алфавита
dropped = df.drop(df[df.word.str.contains(r'[^А-ЯЁа-яё\- ]') ].index)
dropped

In [ ]:
#чистим данные от слишком коротких записей
dropped = dropped.drop(dropped[dropped["word"].apply(lambda x:0 if len(x)== 1 or len(x)==2 else x) == 0].index)
dropped


In [ ]:
#группируем по изделию и формируем список классов для изделия
materials_df = dropped.groupby('word')['class'].apply(list).reset_index(name='классы')
materials_df['классы'] = materials_df['классы'].apply(lambda x: list(set(x)))
materials_df.to_csv("materials in classes.csv")


In [ ]:
#генерируем словарь 
mydict = dict(zip(materials_df["word"], materials_df['классы']))
mydict

In [ ]:
with open("/content/class_product.txt", 'r') as f:
  data_txt = f.read()
result = ast.literal_eval(data_txt)

In [ ]:
#записываем в старый текстовый файл те изделия и классы, которых не хватает
for key,value in mydict.items():
  if key not in result:
    result[key]= value
  else:
      for i in value:
          if i in result[key]:
            continue
          else:
              result[key].append(i)

In [ ]:
#сохраняем результат
result_txt = json.dumps(result, ensure_ascii=False)

with open("class_product.txt", "w", encoding= 'utf-8') as text_file:
    text_file.write(result_txt)

with open("class_product.json", "w", encoding= "utf-8") as fp:
    json.dump(result, fp, ensure_ascii=False) 

**label_enrichment**


---
текстовый файл, в котором указаны теги, для которых будет осуществляться value_augmentation()/агументация, необходимая для того, чтобы модель распознавала эти теги при разных числовых значениях и не привязывалась к конкретным числам

In [ ]:
concat_df = pd.read_csv("/content/labels_and_classes_with marks.csv")

In [ ]:
concat_df['word'] = concat_df['word'].apply(lambda x: x.replace(",", '.'))

In [ ]:
type(concat_df.iloc[2,1])

str

In [ ]:
# concat_df = concat_df[]
concat_df['Num'] = concat_df['word'].apply(lambda x: 1 if is_number(pd.to_numeric(x, errors='ignore')) else 0 )


In [ ]:
num_data = concat_df[concat_df['Num'] == 1]

num_data = num_data[['label']]
num_data = num_data.drop_duplicates(ignore_index=True)
num_data

In [ ]:
with open("/content/labels_enrichment.txt", 'r') as f:
  data_txt = f.readlines()
  result = [line.rstrip() for line in data_txt]


In [ ]:
deleted_tags = ["НД основной", "климатическое исполнение", "длина", 'ширина', 'высота', 'толщина',
                'Артикул', 'толщина покрытия', 'вес', 'вес/грузоподъемность', 'грузоподъемность', 'масса/объем', 
                'сезон/темп.режим', 'категория', "коррозия (припуск/защита)"
                ]
del_marked = []
for i in deleted_tags:
  del_marked.append('B-'+i)
  del_marked.append('I-'+i)
  
  if i == deleted_tags[-1]:
     break



In [ ]:
num_list = num_data['label'].to_list()


In [ ]:

for w in num_list:
  if w in del_marked:
    num_list.remove(w)


далее необходимо еще исключить основные теги из списка, а также те, которые не нуждаются в аугментации (для этого нужна более подробная аналитика показателей по тегам)

In [ ]:
num_list